In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os

# define number of files for all sets
train = 0.8 # 80% of all sound should be in the train set
val = 0.1 # 10% validation set
test = 0.1 # 10% test set
kfolds = 1
basePath="gdrive/My Drive/Indian_birds/Dataset/" # path with sound files
imPath = "gdrive/My Drive/Indian_birds/mels_class/" # path with images (melspectrograms)
destPath = "gdrive/My Drive/Indian_birds/Final_data/" # destination path - where the split dataset should be copied
                            # This folder will be used to train CNNs

# first find all of the mp3 files in the directory
birds=[] # list of all bird spiecies (Ember, Phyll...)
singleBirdList=[] # list of files for one bird
allFilesList=[] # list of all files for all birds. A list of singleBirdLists. 
for root, dirs, files in os.walk(basePath):
    if root == basePath:
        birds=dirs
print(birds)

trainSet=[]
testSet=[]
valSet=[]

birdsShort=[] # list of short file names
birdNumber=0
for nr,bird in enumerate(birds):
    for root, dirs, files in os.walk(basePath+bird):
        for file in files:
            if file.endswith(".mp3"):
                singleBirdList.append(os.path.join(root, file))
    if len(singleBirdList) > 1:
        birdsShort.append(str(birdNumber)+bird[:5])
        birdNumber = birdNumber+1;
        print("Found ", len(singleBirdList), ' mp3 files for ', bird)
        trainSet.append(int(train*len(singleBirdList)))
        valSet.append(int(val*len(singleBirdList)))
        roundDiff = len(singleBirdList)-(int(train*len(singleBirdList))+int(test*len(singleBirdList))+int(val*len(singleBirdList))) 
        testSet.append(int(test*len(singleBirdList)) + roundDiff)
        print("Size of train: ", int(train*len(singleBirdList)),", val: ",int(val*len(singleBirdList)) ,", test: ",int(test*len(singleBirdList)))
        allFilesList.append(singleBirdList)
    singleBirdList=[]


print(trainSet)
print(valSet)
print(testSet)


['Asian koel', 'Common tailorbird', 'White-throated kingfisher', 'White-cheeked barbet', 'Red-vented bulbul', 'Common hawk-cuckoo']
Found  124  mp3 files for  Asian koel
Size of train:  99 , val:  12 , test:  12
Found  126  mp3 files for  Common tailorbird
Size of train:  100 , val:  12 , test:  12
Found  95  mp3 files for  White-throated kingfisher
Size of train:  76 , val:  9 , test:  9
Found  94  mp3 files for  White-cheeked barbet
Size of train:  75 , val:  9 , test:  9
Found  85  mp3 files for  Red-vented bulbul
Size of train:  68 , val:  8 , test:  8
Found  84  mp3 files for  Common hawk-cuckoo
Size of train:  67 , val:  8 , test:  8
[99, 100, 76, 75, 68, 67]
[12, 12, 9, 9, 8, 8]
[13, 14, 10, 10, 9, 9]


In [ ]:
# randomly choose mp3 files for each set  

from random import sample

trainFiles=[]
valFiles=[]
testFiles=[]

for index, singleBirdList in enumerate(allFilesList):
    randFiles = sample(range(len(singleBirdList)), len(singleBirdList))
    start = 0
    end = trainSet[index]
    trainFiles.append(randFiles[start:end])
    start = end 
    end = start + valSet[index]
    valFiles.append(randFiles[start:end])
    start = end 
    end = start + testSet[index] 
    testFiles.append(randFiles[start:end])
    print("Selected random files number:\n train: ", len(trainFiles[index]),"/",trainSet[index],
          ", val: ",len(valFiles[index]),"/",valSet[index],
          ", test: ",len(testFiles[index]),"/",testSet[index])

Selected random files number:
 train:  99 / 99 , val:  12 / 12 , test:  13 / 13
Selected random files number:
 train:  100 / 100 , val:  12 / 12 , test:  14 / 14
Selected random files number:
 train:  76 / 76 , val:  9 / 9 , test:  10 / 10
Selected random files number:
 train:  75 / 75 , val:  9 / 9 , test:  10 / 10
Selected random files number:
 train:  68 / 68 , val:  8 / 8 , test:  9 / 9
Selected random files number:
 train:  67 / 67 , val:  8 / 8 , test:  9 / 9


In [ ]:
def extractName(string):
    return string.rsplit('/',1)[1].replace(' ', '')[:-4]

In [ ]:
# sort all the lists to make copying files easier
sets=[trainFiles,valFiles,testFiles]
for fileSet in sets:
    for index,files in enumerate(fileSet):
        fileSet[index].sort()

In [ ]:
# change full names to short

for root, dirs, files in os.walk(basePath):
    if root == basePath:
        birds=dirs
#birdsShort=[]
#for bird in birds:
#    birdsShort.append(bird[:5])

setNames = ["train/","val/","test/"]

print("Long: ", birds,"\nShort: ",birdsShort)


Long:  ['Asian koel', 'Common tailorbird', 'White-throated kingfisher', 'White-cheeked barbet', 'Red-vented bulbul', 'Common hawk-cuckoo'] 
Short:  ['0Asian', '1Commo', '2White', '3White', '4Red-v', '5Commo']


In [ ]:

import shutil

counter=0
for birdNumber, bird in enumerate(birdsShort): # for each class (bird) check where the file should be copied
    print(counter)
    counter=0
    for setName, fileSet in zip(setNames, sets): # check for all datasets: train, val and test sests 
        for setNumber in fileSet[birdNumber]:
            
            for fileNumber, file in enumerate(allFilesList[birdNumber]):
                if setNumber == fileNumber: # if file number to copy is same as number of file, then copy it
                    
                    for root, dirs, files in os.walk(imPath):
                      
                        for file2 in files:
                      
                            if extractName(file) in file2:
                              
                                counter=counter+1
                                source=root+"/"+file2
                                
                                destination = destPath+setName+bird+"/"
                                if not os.path.exists(destination):
                                    os.makedirs(destination)
                                shutil.copy2(source, destination)
                                #print(source, "   ->   ", destination)

0
944
585
487
574
626
